In [12]:
import pandas as pd
import spacy
from gradio_client import Client

# Load the en_core_web_md model
nlp = spacy.load("en_core_web_md")

# Load student answers and teachers answers (including keywords and marks)
student_df = pd.read_csv('student_answers.csv')
teachers_df = pd.read_csv('teachers_answers.csv')
questions_df = pd.read_csv('questions.csv')

# Initialize a column for calculated marks
student_df['marks_calculated'] = 0

# Loop through each student answer and calculate marks based on keywords or synonyms
for index, student_row in student_df.iterrows():
    question = student_row['question']
    student_answer = student_row['answer']
    
    # Find the corresponding teacher's answer and keywords
    teacher_row = teachers_df[teachers_df['question'] == question].iloc[0]
    teacher_answer = teacher_row['answer']
    teacher_keywords = teacher_row['answer'].split('; ')
    question_max_marks = teacher_row['marks']

    # Check if student's answer is an exact match with teacher's answer
    if student_answer.lower() == teacher_answer.lower():
        student_df.at[index, 'marks_calculated'] = question_max_marks
    else:
        # Calculate marks based on the presence of keywords or synonyms
        student_marks = 0
    
        # Calculate marks based on the presence of keywords or similar words/phrases
        student_marks = 0
        for keyword in teacher_keywords:
            keyword_doc = nlp(keyword)
            student_answer_doc = nlp(student_answer)
            
            # Calculate the similarity between the keyword and student's answer
            similarity = keyword_doc.similarity(student_answer_doc)
            
            # You can adjust the similarity threshold as needed
            if similarity > 0.7:
                student_marks += 1
        
        # Assign calculated marks while ensuring they don't exceed the max marks
        student_df.at[index, 'marks_calculated'] = min(student_marks, question_max_marks)

# Print the updated DataFrame with calculated marks
print(student_df[['question', 'answer', 'marks_calculated']])
print(questions_df)

# Ask the initial message prompt
initial_message = (
    "The message should be to look at each of the teacher's answers points that are separated by a semi-colon (\":\"), "
    "and if it matches the student's answer, then the student is awarded one mark. Each point separated by \";\" is one mark. "
    "So if the student's answers match the teachers answers, then the marks are awarded, and at the end, show how many marks over "
    "the total marks, the student got."
)

# Initialize the Gradio client
client = Client("https://ysharma-explore-llamav2-with-tgi.hf.space/")

# Predict using the Gradio client for the initial message
result_initial = client.predict(
    initial_message,   # str in 'Message' Textbox component
    api_name="/chat"
)
print(result_initial)

# Get student's and teacher's answers for the second message
teacher_answers_text = "* " + "\n* ".join(teachers_df['answer'])
student_answers_text = "* " + "\n* ".join(student_df['answer'])

# Construct the second message
second_message = f"Teacher's Answers:\n{teacher_answers_text}\n\nStudent's Answers:\n{student_answers_text}"

# Predict using the Gradio client for the second message
result_second = client.predict(
    second_message,   # str with formatted student and teacher answers
    api_name="/chat"
)
print(result_second)


   question                                             answer  \
0         1  Interdisciplinary field; Involves design, cons...   
1         2  Intelligence demonstrated by machines; Simulat...   
2         3  Automation of repetitive tasks; Increased prec...   
3         4  Subset of AI, focuses on learning from data; E...   
4         5  Job displacement; Bias in algorithms; Privacy ...   
5         6  Detect environmental changes; Provide feedback...   
6         7  Enables robots to interpret visual information...   
7         8  Narrow AI (Weak AI): Focused on specific tasks...   
8         9  Sensors gather data from surroundings; Process...   
9        10  Mimicking human movement and balance; Replicat...   

   marks_calculated  
0                 5  
1                 8  
2                 6  
3                 7  
4                 9  
5                 4  
6                 6  
7                 7  
8                 5  
9                 7  
                               

In [17]:
# Get student's and teacher's answers for the second message
teacher_answers_text = "* " + "\n* ".join(teachers_df['answer'])
student_answers_text = "* " + "\n* ".join(student_df['answer'])

# Construct the second message
second_message = f"Teacher's Answers:\n{teacher_answers_text}\n\nStudent's Answers:\n{student_answers_text}"

# Predict using the Gradio client for the second message
result_second = client.predict(
    second_message,   # str with formatted student and teacher answers
    api_name="/chat"
)
print(result_second)


Input validation error: `inputs` must have less than 4096 tokens. Given: 5251. Please refresh and try again with an appropriate smaller input prompt.


In [6]:

from gradio_client import Client

client = Client("https://ysharma-explore-llamav2-with-tgi.hf.space/")
result = client.predict(
				"Mark the following student's answer based on the teacher's answers, and assign marks for each point in teacher's answers that is same or similar with the student's answer point",	# str in 'Message' Textbox component
				api_name="/chat"
)
print(result)

Loaded as API: https://ysharma-explore-llamav2-with-tgi.hf.space/ ✔
 Hello! I'm here to help you with any questions or concerns you may have. Please feel free to ask me anything, and I'll do my best to provide a helpful and respectful response. If a question doesn't make sense or is not factually coherent, I'll do my best to explain why instead of providing an incorrect answer. If I don't know the answer to a question, I'll be honest and let you know instead of sharing false information. Let's work together to have a positive and productive conversation!!</s>


In [3]:
# This code marks the student's answers based on the teachers answers. 
import pandas as pd

# Load student answers and teachers answers (including keywords and marks)
student_df = pd.read_csv('student_answers.csv')
teachers_df = pd.read_csv('teachers_answers.csv')
questions_df = pd.read_csv('questions.csv')


# Initialize a column for calculated marks
student_df['marks_calculated'] = 0

# Loop through each student answer and calculate marks based on keywords
for index, student_row in student_df.iterrows():
    question = student_row['question']
    student_answer = student_row['answer']
    
    # Find the corresponding teacher's answer and keywords
    teacher_row = teachers_df[teachers_df['question'] == question].iloc[0]
    teacher_keywords = teacher_row['answer'].split('; ')
    question_max_marks = teacher_row['marks']
    
    # Calculate marks based on the presence of keywords
    student_marks = 0
    for keyword in teacher_keywords:
        if keyword.lower() in student_answer.lower():
            student_marks += 1
    
    # Assign calculated marks while ensuring they don't exceed the max marks
    student_df.at[index, 'marks_calculated'] = min(student_marks, question_max_marks)


#print the questions
print(questions_df)

# Print the updated DataFrame with calculated marks
print(student_df[['question', 'answer', 'marks_calculated']])

                                            question  marks
0                                  What is robotics?      5
1    Explain the concept of artificial intelligence.      8
2    How are robots used in manufacturing processes?      6
3  Discuss the importance of machine learning in AI.      7
4  What are the ethical implications of AI techno...      9
5          Describe the role of sensors in robotics.      4
6      What is computer vision and its applications?      6
7  Explain the difference between narrow AI and g...      7
8       How do robots interact with the environment?      5
9  Discuss the challenges in creating humanoid ro...      8
   question                                             answer  \
0         1  Interdisciplinary field; Involves design, cons...   
1         2  Intelligence demonstrated by machines; Simulat...   
2         3  Automation of repetitive tasks; Increased prec...   
3         4  Subset of AI, focuses on learning from data; E...   
4         

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load data (replace with your data)
data = pd.read_csv('student_answers.csv')


In [5]:

# Preprocess the data (replace with your preprocessing steps)
X = data['answer']
y = data['correct']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:

# Use TF-IDF vectorization (replace with your preferred vectorization method)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [7]:
# Initialize and train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_vec, y_train)


RandomForestClassifier(random_state=42)

In [9]:
# Make predictions
y_pred = clf.predict(X_test_vec)

# Calculate accuracy and print classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print(classification_report(y_test, y_pred))


# Calculate marks obtained for each question
data['marks_obtained'] = y_pred * data['marks']

# Print the updated dataset with marks obtained
print(data)

Accuracy: 1.00
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

   question                                              marks  \
1         1                     What is the capital of France?   
2         1                    Solve the equation: 2x + 5 = 15   
3         2  Discuss the impact of climate change on biodiv...   

                                              answer  correct  \
1                                              Paris        1   
2                                              x = 5        1   
3  Climate change has significant effects on biod...        1   

                                      marks_obtained  
1                     What is the capital of France?  
2                    Solve the equation: 2x + 5 = 15  
3  Discuss the impact